### softmax回归的简洁实现

In [1]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import torchvision
import sys
import utils

#### 获取和读取数据

In [2]:
batch_size=256
train_iter,test_iter=utils.load_data_fashion_mnist(batch_size)

#### 定义和初始化模型

因为前面我们数据返回的每个batch样本x的形状为(batch_size,1,28,28),所以我们要先用view()将x的形状转换成(batch_size,784)才送入全连接层。

In [3]:
num_inputs=784
num_outputs=10

class LinearNet(nn.Module):
    def __init__(self,num_inputs,num_outputs):
        super(LinearNet,self).__init__()
        self.linear=nn.Linear(num_inputs,num_outputs)
    def forward(self,x): #x shape:(batsh_size,1,28,28)
        y=self.linear(x.view(x.shape[0],-1))
        return y

In [4]:
net=LinearNet(num_inputs,num_outputs)

我们将对x的形状转换的这个功能自定义一个FlattenLayer并记录在utils中方便后面使用。

In [5]:
class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer,self).__init__()
    def forward(self,x): # x shape:(batch,*,*,...)
        return x.view(x.shape[0],-1)

这样就可以更方便地定义模型

In [6]:
net1=nn.Sequential(
    FlattenLayer(),
    nn.Linear(num_inputs,num_outputs)
)

In [7]:
#初始化参数
init.normal_(net.linear.weight,mean=0,std=0.01)
init.constant_(net.linear.bias,val=0)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

#### softmax和交叉熵损失函数

Pytorch提供了一个包含softmax运算和交叉熵损失计算的函数CrossEntropyLoss(),它的数值稳定性更好。

In [8]:
loss=nn.CrossEntropyLoss()

#### 定义优化算法

In [9]:
optimizer=torch.optim.SGD(net.parameters(),lr=0.1)

#### 训练模型

In [10]:
num_epochs=5
utils.train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,None,None,optimizer)

epoch 1,loss 0.0031,train acc0.750,test acc 0.787
epoch 2,loss 0.0022,train acc0.813,test acc 0.796
epoch 3,loss 0.0021,train acc0.825,test acc 0.809
epoch 4,loss 0.0020,train acc0.832,test acc 0.813
epoch 5,loss 0.0019,train acc0.836,test acc 0.816
